In [1]:
import pandas as pd
import pprint
import numpy as np
import collections
import sklearn.preprocessing as preprocessing
from IPython.core.debugger import Tracer
import json
import operator

# TODO: Temos um erro no modelo, que é usar o labelEncoder na coluna de segmento, se estivermos
# implementando um modelo de ML, isso seria um problema, o modelo 'pensaria' que as categorias com
# maior valor são mais importantes, quando na verdade, têm a mesma importância

# Como estamos usando uma lógica própria e não um algoritmo de ML, esse problema não impacta
# A comparação que fazemos é se o segmento do usuario a receber a recomendação é igual ao segmento
# do usuário que poderá ser recomendado

In [2]:
dados_json = pd.read_json("dados.json")

# columns = reversed(dados_json.columns)
dados_json['user_id'] = range(0,len(dados_json))
dados_json = dados_json.reindex_axis(['user_id','tipo_user','habilidades','segmento','horas_disponiveis'], axis=1) #reordenamos somente pra melhorar a visualização e ficar igual csv
dados_json.to_json(path_or_buf='./dados_.json',orient='records')
dados_json.head()
# print(dados_json.iloc[137])
dados_json.size

690

In [77]:
for item in dados_json.itertuples():

    indexes = []
    #condicao = len(dados_json.iloc[item.Index]['habilidades']) == 1 and len(dados_json.iloc[item.Index]['habilidades'][0]) == 0
    print(item.Index)
#     if condicao:
#         print(item.Index)
#         print(dados_json.iloc[item.Index])
#         indexes.append(item.Index)
#         print(50*"*")
        


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
44
45
46
49
51
52
53
54
55
56
59
62
73
75
77
80
81
83
84
85
86
88
89
92
93
95
96
97
99
101
102
103
104
105
106
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137


In [3]:
dados_json

,user_id,tipo_user,habilidades,segmento,horas_disponiveis
0,0,startup,"[python, php, js, scrum]",fintech,6
1,1,user,"[python, js, illustrator]",edtech,8
2,2,startup,"[laravel, php, js, html]",biotech,4
3,3,startup,"[scrum, laravel, angular, html]",adtech,6
4,4,startup,"[js, html, css]",adtech,8
5,5,startup,"[js, angular, ionic, html, css]",fintech,4
6,6,startup,"[php, laravel, scrum, js, html]",fintech,4
7,7,user,"[js, angular, ionic, html, css]",edtech,5
8,8,startup,"[js, html, css, illustrator]",edtech,5
9,9,startup,"[scrum, laravel, angular, html]",healthtech,8


In [4]:
classes = set([x.strip() for y in dados_json['habilidades'] for x in y])
classes

{'',
 'angular',
 'big-data',
 'branding',
 'cakephp',
 'css',
 'gestao-projetos',
 'html',
 'illustrator',
 'ionic',
 'js',
 'laravel',
 'machine-learning',
 'marketing-digital',
 'photoshop',
 'php',
 'python',
 'react',
 'scikit-learn',
 'scrum',
 'scss',
 'seo'}

In [5]:
dados_json = pd.read_json("dados.json")

# columns = reversed(dados_json.columns)
dados_json['user_id'] = range(0,len(dados_json))
dados_json = dados_json.reindex_axis(['user_id','tipo_user','habilidades','segmento','horas_disponiveis'], axis=1) #reordenamos somente pra melhorar a visualização e ficar igual csv

classes = set([x.strip() for y in dados_json['habilidades'] for x in y])
#aqui ainda nao existe uma coluna pra cada classe
#criando as colunas com o nome das classes
for classe in classes: dados_json[classe] = 0
    
# # Modifying columns to '1' when habilidade is found
   
for i in range(len(dados_json['habilidades'])): # para cada linha, ou seja, cada usuario
    classes_line = dados_json.loc[i,'habilidades'] #pegamos suas habilidades
#     print(classes_line);
#     print("-------------")
    for j in range(len(classes_line)): #para cada habilidade daquele usuario
        classe = classes_line[j] #pegamos a atual habilidade do usuario
        if classe in classes: #verificamos se a habilidade atual existe no nosso vetor de classes global
            dados_json.loc[:, (classe)][i] = 1 #aqui seria esse warning -> mas n conseguimos resolver
              
del dados_json['habilidades']
dados_json.head()
# print(classes)
# dados_json

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,user_id,tipo_user,segmento,horas_disponiveis,,gestao-projetos,python,seo,laravel,photoshop,...,big-data,angular,html,scss,js,ionic,illustrator,cakephp,css,react
0,0,startup,fintech,6,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,1,user,edtech,8,0,0,1,0,0,0,...,0,0,0,0,1,0,1,0,0,0
2,2,startup,biotech,4,0,0,0,0,1,0,...,0,0,1,0,1,0,0,0,0,0
3,3,startup,adtech,6,0,0,0,0,1,0,...,0,1,1,0,0,0,0,0,0,0
4,4,startup,adtech,8,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,1,0


In [6]:
#Conferindo formato csv
dados_csv = pd.read_csv("dados.csv",sep=";") 
dados_csv.head()

,tipo_user,habilidades,segmento,horas_disponiveis
0,startup,"python,php,js,scrum",fintech,6
1,user,"python,js,illustrator",edtech,8
2,startup,"laravel, php, js, html",biotech,4
3,startup,"scrum,laravel,angular,html",adtech,6
4,startup,"js,html,css",adtech,8


In [7]:
dados_json.head()

,user_id,tipo_user,segmento,horas_disponiveis,,gestao-projetos,python,seo,laravel,photoshop,...,big-data,angular,html,scss,js,ionic,illustrator,cakephp,css,react
0,0,startup,fintech,6,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,1,user,edtech,8,0,0,1,0,0,0,...,0,0,0,0,1,0,1,0,0,0
2,2,startup,biotech,4,0,0,0,0,1,0,...,0,0,1,0,1,0,0,0,0,0
3,3,startup,adtech,6,0,0,0,0,1,0,...,0,1,1,0,0,0,0,0,0,0
4,4,startup,adtech,8,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,1,0


In [8]:
class Ranking (object):

    def __init__(self,df,preprocessed=True):
        self.df = df
        
    def getDf (self):
        return self.df
    
    def preprocessDataFrame (self):# testar se é preciso receber df como parametro
        categorical_columns = self.df.select_dtypes(['object']).columns
        label_encoder = preprocessing.LabelEncoder()
        for column in categorical_columns:
            label_encoder.fit(self.df[column])
            label_encoder.classes_
            self.df[column] = label_encoder.transform(self.df[column])
    
    def train_test_split (self,test_size=0.6):
        limit = int((len(self.df)*test_size))
        X_treino = self.df[:limit]
        X_teste = self.df[limit:].reset_index(drop=True)
        return X_treino, X_teste
    
    
    '''
        Função que gera um dicionario de rankings dado um conjunto de teste(à ser recomendado) 
        e um conjunto de treinamento para realizar a recomendação
    '''
    # Do jeito que rank_the_data está hoje, estamos sujeitos à ter que fazer manutenção constante no código
    # Somente na parte dos pesos, talvez tornamos ele um dicionario cuja chave seja o nome da coluna com seu valor
    # o peso respectivo => TODO <=
    def rank_the_data(self,X_treino,X_teste, name_treino='Ranking_Treino', name_teste='Ranking_Teste'):

        temp_matches = {}
        matches_dict = {}
        
        for row in X_teste.itertuples(name=name_teste): # definindo a row atual do x_teste
            match = False
            valores_teste  = []
            valores_treino = []
            rating = 0
            pesos  = (len(row) - 3) * [0.0] #array com os pesos

            pesos[0] = 0.7 # aumentando peso do segmento para quando houver match

            for i in range(2,len(pesos)):
                pesos[i] = 1.0 # aumentando o peso das habilidades
            # para a disponibilidade de horas nós temos que fazer um calculo mais detalhado
            # diponibilidade_startup > disponibilidade_user; disponibilidade_user/disponibilidade_startup => peso
            # diponibilidade_startup < disponibilidade_user; disponibilidade_startup/disponibilidade_user => peso
#             Tracer()()
            for i in range(3,len(row)):  
            # aqui é o loop do item da linha (colunas) começamos do index 2 por causa da estrutura da tupla
                valores_teste.append(row[i])

            for row_treino in X_treino.itertuples(name=name_treino):# loop da row atual do x_treino
                for i_treino in range(3, len(row_treino)): # loop dos itens da linha (colunas)
                    valores_treino.append(row_treino[i_treino]) # valor da coluna

                if not(row.tipo_user == row_treino.tipo_user): # nao podemos computar valores para usuarios de mesmo tipo
                    # para adicionar funcionalidade de peso teremos que modificar a partir da current_sum
                    # pensamento: e se já de inicio nós tivermos um array com todos os pesos settados com 1 e fizermos
                    # a multiplicação

                    #peso disp_horas
                    if row.horas_disponiveis > row_treino.horas_disponiveis:
                        pesos[1] = row_treino.horas_disponiveis/row.horas_disponiveis
                    elif row.horas_disponiveis == row_treino.horas_disponiveis:
                        pesos[1] = 0.75 # se o horário for igual devemos aumentar o peso padrão
                    else:
                        pesos[1] = row.horas_disponiveis/row_treino.horas_disponiveis

                    current_sum = (sum(np.equal(valores_teste, valores_treino)*pesos)+ pesos[1])/(len(X_teste.columns) - 2)
                    rating = np.around(current_sum, decimals=2)
                    temp_matches[row_treino] = float(format(rating, '.2f'))
                valores_treino = [] #resetamos o valor a cada iteração que a row de treinamento muda

            matches_dict[row] = temp_matches
            temp_matches = {}
        return matches_dict
    
    def filter_ranked_data (self,matches_dict, min_val=0.65):
        results = {}
        for key,value in matches_dict.items():
            filtered = {k:v for k,v in value.items() if v >= min_val}
            
            for k,v in filtered.items():
                results[key.user_id] = {k.user_id:v for k,v in filtered.items()}

        return results #retornamos um dicionario com os dados filtrados, isso é o que temos que mandar através da api e tratar
    
    def visualize_recommendations(self,filtered_dict, categoric_df):
    
        for key, value in filtered_dict.items():
            test_data = categoric_df.loc[key]
            columns = categoric_df.columns
            print("-----------------------")
            print("Dado para teste: ")
            print("-----------------------")
            for i in range(len(test_data)):
                print("{0}: {1}".format(columns[i], test_data[i]))
            print("---------------------------------------------")
            print("Recomendações: ")
            print("-----------------------")

            value = collections.OrderedDict(sorted(value.items(),key=lambda x: x[1],reverse=True)) #ordenando por valores(ratings)
            
            for k,v in value.items():
                rating = v
                rating_user = categoric_df.loc[k]
                print("Recomendação: %.2f%%" % (rating*100))
                for i in range(len(rating_user)):
                    print("{0}: {1}".format(columns[i], rating_user[i]))
                print("----")

In [9]:
dados_raw = pd.read_json("dados.json")
rank      = Ranking(dados_json)
rank.preprocessDataFrame()
X_treino  = dados_json[:len(dados_json)-2] 
X_teste   = dados_json[len(dados_json)-2:]
ranked_data = rank.rank_the_data(X_treino, X_teste)
filtered_data = rank.filter_ranked_data(ranked_data,min_val=0.80) #retorna a % e o id do usuario relacionado
print(X_teste)

     user_id  tipo_user  segmento  horas_disponiveis     gestao-projetos  \
136      136          0         0                  8  0                0   
137      137          1         6                  4  0                0   

     python  seo  laravel  photoshop  ...    big-data  angular  html  scss  \
136       0    0        0          0  ...           0        0     0     0   
137       0    1        0          0  ...           0        1     0     0   

     js  ionic  illustrator  cakephp  css  react  
136   0      0            0        1    0      0  
137   0      1            0        0    0      0  

[2 rows x 26 columns]


In [6]:
rank.visualize_recommendations(filtered_data, dados_raw)

-----------------------
Dado para teste: 
-----------------------
habilidades: ['machine-learning', 'php', 'cakephp']
horas_disponiveis: 8
segmento: adtech
tipo_user: startup
---------------------------------------------
Recomendações: 
-----------------------
Recomendação: 88.00%
habilidades: ['machine-learning', 'angular']
horas_disponiveis: 8
segmento: adtech
tipo_user: user
----
Recomendação: 85.00%
habilidades: ['php', 'marketing-digital']
horas_disponiveis: 8
segmento: edtech
tipo_user: user
----
Recomendação: 85.00%
habilidades: ['machine-learning', 'ionic', 'js', 'php']
horas_disponiveis: 6
segmento: adtech
tipo_user: user
----
Recomendação: 85.00%
habilidades: ['cakephp']
horas_disponiveis: 4
segmento: biotech
tipo_user: user
----
Recomendação: 84.00%
habilidades: ['']
horas_disponiveis: 8
segmento: adtech
tipo_user: user
----
Recomendação: 84.00%
habilidades: ['css']
horas_disponiveis: 8
segmento: adtech
tipo_user: user
----
Recomendação: 81.00%
habilidades: ['big-data']
hora

In [20]:
def generateFullJSON(raw_df,filtered_dict):
    #the JSON should return the following structure:
    jsons      = []
    for key,item in filtered_dict.items():
        user_type = raw_df.loc[:,'tipo_user'][key]
        user_id = key
        skills = raw_df.loc[:,'habilidades'][key]
        market = raw_df.loc[:, 'segmento'][key]
        hours = raw_df.loc[:, 'horas_disponiveis'][key]
        codigo = collections.OrderedDict({"user_id": int(user_id), "user_type": user_type, 
                  "skills":skills,"market":market,"hours":float(hours),"ratings": []})
        
        #ordenando pela rating
        ordered_dict = collections.OrderedDict(sorted(item.items(), key= lambda x:x[1],reverse=True))
        #print("Ordered dict", ordered_dict)
        for k,v in ordered_dict.items():
            
            guess_user_type = raw_df.loc[:, 'tipo_user'][k]
            guess_user_id = k
            guess_skills = raw_df.loc[:,'habilidades'][k]
            guess_market = raw_df.loc[:, 'segmento'][k]
            guess_hours = raw_df.loc[:, 'horas_disponiveis'][k]
            
            codigo["ratings"].append(collections.OrderedDict({"user_id": int(guess_user_id), "user_type": guess_user_type, 
                  "skills":guess_skills,"market":guess_market,"hours":float(guess_hours),"rating":float(v)}))
            
        jsons.append(codigo)
    return json.dumps(jsons)
            
        
    

In [21]:
JSON = generateFullJSON(dados_raw, filtered_data)
JSON

'[{"user_type": "startup", "skills": ["machine-learning", "php", "cakephp"], "market": "adtech", "hours": 8.0, "ratings": [{"rating": 0.9, "user_type": "user", "skills": ["machine-learning", "angular"], "market": "adtech", "hours": 8.0, "user_id": 59}, {"rating": 0.87, "user_type": "user", "skills": ["php", "marketing-digital"], "market": "edtech", "hours": 8.0, "user_id": 67}, {"rating": 0.86, "user_type": "user", "skills": [""], "market": "adtech", "hours": 8.0, "user_id": 66}, {"rating": 0.86, "user_type": "user", "skills": ["machine-learning", "ionic", "js", "php"], "market": "adtech", "hours": 6.0, "user_id": 15}, {"rating": 0.86, "user_type": "user", "skills": ["css"], "market": "adtech", "hours": 8.0, "user_id": 92}, {"rating": 0.85, "user_type": "user", "skills": ["cakephp"], "market": "biotech", "hours": 4.0, "user_id": 94}, {"rating": 0.82, "user_type": "user", "skills": ["big-data", "machine-learning", "scikit-learn"], "market": "healthtech", "hours": 8.0, "user_id": 27}, {"

In [104]:
pesos ={}

for column in X_treino.columns[2:]:
    pesos[column] = 0
pesos

{'angular': 0,
 'big-data': 0,
 'css': 0,
 'horas_disponiveis': 0,
 'html': 0,
 'illustrator': 0,
 'ionic': 0,
 'js': 0,
 'laravel': 0,
 'machine-learning': 0,
 'photoshop': 0,
 'php': 0,
 'python': 0,
 'scikit-learn': 0,
 'scrum': 0,
 'scss': 0,
 'segmento': 0}

In [102]:
for peso in pesos :
    if peso not in ['segmento','horas_disponiveis']:
        print(peso)

big-data
machine-learning
illustrator
ionic
python
angular
scikit-learn
html
photoshop
css
scss
js
laravel
php
scrum


In [153]:
a = 0.9400000000000
a = float(format(a, '.2f'))

In [154]:
type(a)

float